# Focused Learning: Error Traceback and Analysis in CoderGen

## Mục tiêu học tập

Notebook này tập trung vào một trong những khái niệm quan trọng nhất của CoderGen framework: **Error Traceback and Analysis** (Section 3.1). Chúng ta sẽ:

1. Hiểu cách CoderGen phân tích lỗi từ code execution
2. Implement error analysis mechanism với pattern matching
3. Xây dựng suggestion system để fix errors
4. Tạo feedback loop giữa error analysis và code generation

## Trích xuất từ paper

**Section 3.1: Error Traceback and Analysis** (trang 5-6)
- "The CoderGen framework includes a robust error traceback and analysis mechanism"
- "The system captures the error traceback, which provides a detailed record of the path through the code that led to the failure"
- Figure 3 shows error traceback analyze example
- "The error analysis component leverages the fine-tuned language model to interpret the error messages and suggest potential fixes"

## 1. Lý thuyết: Error Analysis trong Code Generation

### 1.1. Types of Errors in AI Code Generation

Khi generate code cho AI tasks, có nhiều loại lỗi có thể xảy ra:

1. **Syntax Errors**: Lỗi cú pháp Python
2. **Import Errors**: Missing hoặc incorrect library imports
3. **API Errors**: Wrong API usage, deprecated methods
4. **Runtime Errors**: Type mismatches, null pointers, resource errors
5. **Logic Errors**: Code runs nhưng output sai

### 1.2. Error Traceback Components

Python traceback chứa các thông tin quan trọng:
- **Error Type**: Class của exception (e.g., ValueError, ImportError)
- **Error Message**: Mô tả cụ thể về lỗi
- **Stack Trace**: Call stack showing execution path
- **Line Numbers**: Exact location của error

### 1.3. Analysis Strategy

$$\text{Error Fix} = f(\text{Error Type}, \text{Context}, \text{Domain Knowledge})$$

Trong đó:
- Error Type giúp classify approach
- Context từ code xung quanh
- Domain Knowledge về AI libraries

In [ ]:
import re
import traceback
import sys
from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass, field
from enum import Enum
import json
import ast
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

## 2. Error Classification System

In [ ]:
class ErrorCategory(Enum):
    """Categories of errors in AI code generation"""
    SYNTAX = "syntax_error"
    IMPORT = "import_error"
    API_USAGE = "api_usage_error"
    RUNTIME = "runtime_error"
    LOGIC = "logic_error"
    RESOURCE = "resource_error"
    TYPE = "type_error"

@dataclass
class ErrorInfo:
    """Structured error information"""
    error_type: str
    error_message: str
    file_path: str
    line_number: int
    code_context: List[str]
    stack_trace: List[Dict[str, Any]]
    category: ErrorCategory
    
@dataclass
class ErrorFix:
    """Suggested fix for an error"""
    description: str
    code_changes: List[Dict[str, str]]  # [{"old": "...", "new": "..."}]
    confidence: float
    explanation: str

## 3. Error Parser Implementation

In [ ]:
class ErrorParser:
    """Parse Python error tracebacks into structured format"""
    
    def __init__(self):
        # Regex patterns for parsing tracebacks
        self.traceback_pattern = re.compile(r'File "(.+?)", line (\d+), in (.+)')
        self.error_pattern = re.compile(r'^(\w+Error|\w+Exception): (.+)$', re.MULTILINE)
        
    def parse_traceback(self, traceback_str: str) -> ErrorInfo:
        """Parse a traceback string into ErrorInfo"""
        lines = traceback_str.strip().split('\n')
        
        # Extract error type and message
        error_match = self.error_pattern.search(traceback_str)
        if error_match:
            error_type = error_match.group(1)
            error_message = error_match.group(2)
        else:
            error_type = "UnknownError"
            error_message = lines[-1] if lines else "Unknown error"
        
        # Parse stack trace
        stack_trace = []
        for line in lines:
            match = self.traceback_pattern.search(line)
            if match:
                stack_trace.append({
                    "file": match.group(1),
                    "line": int(match.group(2)),
                    "function": match.group(3)
                })
        
        # Get the most recent error location
        if stack_trace:
            latest_error = stack_trace[-1]
            file_path = latest_error["file"]
            line_number = latest_error["line"]
        else:
            file_path = "unknown"
            line_number = 0
        
        # Extract code context (simplified)
        code_context = self._extract_code_context(traceback_str)
        
        # Categorize error
        category = self._categorize_error(error_type, error_message)
        
        return ErrorInfo(
            error_type=error_type,
            error_message=error_message,
            file_path=file_path,
            line_number=line_number,
            code_context=code_context,
            stack_trace=stack_trace,
            category=category
        )
    
    def _extract_code_context(self, traceback_str: str) -> List[str]:
        """Extract code lines from traceback"""
        context = []
        lines = traceback_str.split('\n')
        
        for i, line in enumerate(lines):
            if line.strip().startswith("    ") and not line.strip().startswith("    ^"):
                context.append(line.strip())
        
        return context
    
    def _categorize_error(self, error_type: str, error_message: str) -> ErrorCategory:
        """Categorize error based on type and message"""
        error_type_lower = error_type.lower()
        error_message_lower = error_message.lower()
        
        if "syntax" in error_type_lower:
            return ErrorCategory.SYNTAX
        elif "import" in error_type_lower or "module" in error_message_lower:
            return ErrorCategory.IMPORT
        elif "type" in error_type_lower:
            return ErrorCategory.TYPE
        elif any(api in error_message_lower for api in ["pipeline", "model", "tokenizer"]):
            return ErrorCategory.API_USAGE
        elif any(resource in error_message_lower for resource in ["file", "path", "directory"]):
            return ErrorCategory.RESOURCE
        else:
            return ErrorCategory.RUNTIME

## 4. AI-Powered Error Analyzer

In [ ]:
class AIErrorAnalyzer:
    """Analyze errors and suggest fixes using AI knowledge"""
    
    def __init__(self):
        # Knowledge base of common errors and fixes
        self.error_patterns = self._build_error_patterns()
        self.api_knowledge = self._build_api_knowledge()
    
    def analyze_error(self, error_info: ErrorInfo, original_code: str) -> List[ErrorFix]:
        """Analyze error and suggest fixes"""
        fixes = []
        
        # Strategy based on error category
        if error_info.category == ErrorCategory.IMPORT:
            fixes.extend(self._analyze_import_error(error_info, original_code))
        elif error_info.category == ErrorCategory.API_USAGE:
            fixes.extend(self._analyze_api_error(error_info, original_code))
        elif error_info.category == ErrorCategory.TYPE:
            fixes.extend(self._analyze_type_error(error_info, original_code))
        elif error_info.category == ErrorCategory.RESOURCE:
            fixes.extend(self._analyze_resource_error(error_info, original_code))
        else:
            fixes.extend(self._analyze_generic_error(error_info, original_code))
        
        # Sort by confidence
        fixes.sort(key=lambda x: x.confidence, reverse=True)
        
        return fixes
    
    def _build_error_patterns(self) -> Dict[str, List[Dict[str, Any]]]:
        """Build knowledge base of error patterns"""
        return {
            "ImportError": [
                {
                    "pattern": "No module named 'transformers'",
                    "fix": "pip install transformers",
                    "code_change": None
                },
                {
                    "pattern": "cannot import name 'pipeline'",
                    "fix": "from transformers import pipeline",
                    "code_change": {"old": "import pipeline", "new": "from transformers import pipeline"}
                }
            ],
            "NameError": [
                {
                    "pattern": "name 'pipeline' is not defined",
                    "fix": "Import pipeline from transformers",
                    "code_change": {"add_import": "from transformers import pipeline"}
                }
            ],
            "TypeError": [
                {
                    "pattern": "expected str, bytes or os.PathLike object",
                    "fix": "Convert input to string",
                    "code_change": {"wrap": "str()"}
                }
            ]
        }
    
    def _build_api_knowledge(self) -> Dict[str, Dict[str, Any]]:
        """Build API usage knowledge base"""
        return {
            "pipeline": {
                "correct_usage": "pipeline('task-name', model='model-name')",
                "common_tasks": ["text-classification", "sentiment-analysis", "ner", "question-answering"],
                "required_imports": ["from transformers import pipeline"]
            },
            "AutoModel": {
                "correct_usage": "AutoModel.from_pretrained('model-name')",
                "variants": ["AutoModelForSequenceClassification", "AutoModelForTokenClassification"],
                "required_imports": ["from transformers import AutoModel"]
            },
            "torch.load": {
                "correct_usage": "torch.load('path/to/model.pt')",
                "common_errors": ["map_location", "pickle errors"],
                "required_imports": ["import torch"]
            }
        }
    
    def _analyze_import_error(self, error_info: ErrorInfo, code: str) -> List[ErrorFix]:
        """Analyze import errors and suggest fixes"""
        fixes = []
        
        # Check for missing module
        if "No module named" in error_info.error_message:
            module_match = re.search(r"No module named '([^']+)'", error_info.error_message)
            if module_match:
                module_name = module_match.group(1)
                
                # Map common module names to packages
                package_mapping = {
                    "transformers": "transformers",
                    "torch": "torch",
                    "torchvision": "torchvision",
                    "PIL": "pillow",
                    "cv2": "opencv-python"
                }
                
                package = package_mapping.get(module_name, module_name)
                
                fixes.append(ErrorFix(
                    description=f"Install missing package '{package}'",
                    code_changes=[{
                        "old": "",
                        "new": f"# Add to requirements: pip install {package}"
                    }],
                    confidence=0.95,
                    explanation=f"The module '{module_name}' is not installed. Install it using pip."
                ))
        
        # Check for incorrect import syntax
        if "cannot import name" in error_info.error_message:
            name_match = re.search(r"cannot import name '([^']+)'", error_info.error_message)
            if name_match:
                import_name = name_match.group(1)
                
                # Suggest correct import based on API knowledge
                for api_name, api_info in self.api_knowledge.items():
                    if import_name == api_name:
                        fixes.append(ErrorFix(
                            description=f"Use correct import for {import_name}",
                            code_changes=[{
                                "old": f"import {import_name}",
                                "new": api_info["required_imports"][0]
                            }],
                            confidence=0.9,
                            explanation=f"The correct way to import {import_name} is: {api_info['required_imports'][0]}"
                        ))
        
        return fixes
    
    def _analyze_api_error(self, error_info: ErrorInfo, code: str) -> List[ErrorFix]:
        """Analyze API usage errors"""
        fixes = []
        
        # Check for pipeline errors
        if "pipeline" in error_info.error_message:
            # Common pipeline error: wrong task name
            if "Unknown task" in error_info.error_message or "not a valid" in error_info.error_message:
                fixes.append(ErrorFix(
                    description="Use valid pipeline task name",
                    code_changes=[{
                        "old": "pipeline('text-classification')",
                        "new": "pipeline('sentiment-analysis')  # or 'text-classification' with model"
                    }],
                    confidence=0.85,
                    explanation="Pipeline requires valid task names. Common tasks: sentiment-analysis, ner, question-answering"
                ))
            
            # Missing model specification
            if "model" in error_info.error_message:
                fixes.append(ErrorFix(
                    description="Specify model for pipeline",
                    code_changes=[{
                        "old": "pipeline('task-name')",
                        "new": "pipeline('task-name', model='model-name')"
                    }],
                    confidence=0.8,
                    explanation="Some tasks require explicit model specification"
                ))
        
        # Check for model loading errors
        if "from_pretrained" in error_info.error_message:
            fixes.append(ErrorFix(
                description="Check model name and availability",
                code_changes=[{
                    "old": "AutoModel.from_pretrained('model-name')",
                    "new": "AutoModel.from_pretrained('bert-base-uncased')  # Use valid model name"
                }],
                confidence=0.75,
                explanation="Ensure the model name exists on HuggingFace Hub"
            ))
        
        return fixes
    
    def _analyze_type_error(self, error_info: ErrorInfo, code: str) -> List[ErrorFix]:
        """Analyze type errors"""
        fixes = []
        
        # String conversion errors
        if "expected str" in error_info.error_message:
            fixes.append(ErrorFix(
                description="Convert input to string",
                code_changes=[{
                    "old": "function(variable)",
                    "new": "function(str(variable))"
                }],
                confidence=0.8,
                explanation="The function expects a string input. Use str() to convert."
            ))
        
        # List/tensor conversion
        if "expected Tensor" in error_info.error_message:
            fixes.append(ErrorFix(
                description="Convert to PyTorch tensor",
                code_changes=[{
                    "old": "model(data)",
                    "new": "model(torch.tensor(data))"
                }],
                confidence=0.85,
                explanation="PyTorch models expect tensor inputs"
            ))
        
        return fixes
    
    def _analyze_resource_error(self, error_info: ErrorInfo, code: str) -> List[ErrorFix]:
        """Analyze resource/file errors"""
        fixes = []
        
        if "FileNotFoundError" in error_info.error_type or "No such file" in error_info.error_message:
            fixes.append(ErrorFix(
                description="Check file path and create if needed",
                code_changes=[{
                    "old": "open('file.txt')",
                    "new": """import os
if not os.path.exists('file.txt'):
    # Create file or download from URL
    raise FileNotFoundError('Please provide the file')
open('file.txt')"""
                }],
                confidence=0.9,
                explanation="The file doesn't exist. Check the path or create it."
            ))
        
        return fixes
    
    def _analyze_generic_error(self, error_info: ErrorInfo, code: str) -> List[ErrorFix]:
        """Generic error analysis"""
        fixes = []
        
        # Try to find pattern matches
        for error_type, patterns in self.error_patterns.items():
            if error_type in error_info.error_type:
                for pattern in patterns:
                    if pattern["pattern"] in error_info.error_message:
                        fix = ErrorFix(
                            description=pattern["fix"],
                            code_changes=[pattern.get("code_change", {"old": "", "new": ""})],
                            confidence=0.7,
                            explanation="Based on error pattern matching"
                        )
                        fixes.append(fix)
        
        # If no specific fixes found, suggest generic debugging
        if not fixes:
            fixes.append(ErrorFix(
                description="Add error handling and debugging",
                code_changes=[{
                    "old": "# Your code",
                    "new": """try:
    # Your code
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()"""
                }],
                confidence=0.5,
                explanation="Add try-except block for better error handling"
            ))
        
        return fixes

## 5. Error-Driven Code Regeneration

In [ ]:
class CodeRegenerator:
    """Regenerate code based on error analysis"""
    
    def __init__(self, analyzer: AIErrorAnalyzer):
        self.analyzer = analyzer
        self.regeneration_history = []
    
    def regenerate_with_fixes(self, original_code: str, error_info: ErrorInfo, 
                             task_description: str) -> Tuple[str, List[str]]:
        """Regenerate code incorporating error fixes"""
        
        # Get suggested fixes
        fixes = self.analyzer.analyze_error(error_info, original_code)
        
        if not fixes:
            return original_code, ["No fixes suggested"]
        
        # Apply top fixes
        modified_code = original_code
        applied_fixes = []
        
        for fix in fixes[:3]:  # Apply top 3 fixes
            if fix.confidence > 0.7:
                modified_code = self._apply_fix(modified_code, fix)
                applied_fixes.append(fix.description)
        
        # Generate improved prompt for LLM
        improved_prompt = self._generate_improved_prompt(
            task_description, error_info, fixes[0]
        )
        
        # Track regeneration
        self.regeneration_history.append({
            "error": error_info,
            "fixes_applied": applied_fixes,
            "original_code": original_code,
            "modified_code": modified_code
        })
        
        return modified_code, applied_fixes
    
    def _apply_fix(self, code: str, fix: ErrorFix) -> str:
        """Apply a specific fix to the code"""
        modified_code = code
        
        for change in fix.code_changes:
            if "old" in change and "new" in change:
                modified_code = modified_code.replace(change["old"], change["new"])
            elif "add_import" in change:
                # Add import at the beginning
                import_line = change["add_import"]
                if import_line not in modified_code:
                    lines = modified_code.split('\n')
                    # Find where to insert import
                    import_idx = 0
                    for i, line in enumerate(lines):
                        if line.startswith('import') or line.startswith('from'):
                            import_idx = i + 1
                    lines.insert(import_idx, import_line)
                    modified_code = '\n'.join(lines)
        
        return modified_code
    
    def _generate_improved_prompt(self, task: str, error: ErrorInfo, 
                                 fix: ErrorFix) -> str:
        """Generate improved prompt based on error analysis"""
        prompt = f"""
Task: {task}

Previous attempt resulted in {error.error_type}: {error.error_message}

Please generate code that:
1. {fix.description}
2. {fix.explanation}
3. Includes proper error handling
4. Uses correct API calls for the libraries

Make sure to:
- Import all necessary modules
- Handle edge cases
- Follow the library's best practices
"""
        return prompt

## 6. Complete Error Analysis Pipeline Demo

In [ ]:
# Initialize components
parser = ErrorParser()
analyzer = AIErrorAnalyzer()
regenerator = CodeRegenerator(analyzer)

# Example 1: Import Error
print("=== Example 1: Import Error ===")
traceback_1 = """Traceback (most recent call last):
  File "test.py", line 3, in <module>
    classifier = pipeline('text-classification')
NameError: name 'pipeline' is not defined"""

code_1 = """# Text classification example
def classify_text(text):
    classifier = pipeline('text-classification')
    return classifier(text)
"""

error_info_1 = parser.parse_traceback(traceback_1)
print(f"Error Type: {error_info_1.error_type}")
print(f"Category: {error_info_1.category.value}")
print(f"Line Number: {error_info_1.line_number}")

fixes_1 = analyzer.analyze_error(error_info_1, code_1)
print(f"\nSuggested Fixes ({len(fixes_1)}):")
for i, fix in enumerate(fixes_1[:3]):
    print(f"{i+1}. {fix.description} (confidence: {fix.confidence:.2f})")
    print(f"   Explanation: {fix.explanation}")

# Apply fixes
fixed_code_1, applied_1 = regenerator.regenerate_with_fixes(
    code_1, error_info_1, "Create text classification function"
)
print("\nFixed Code:")
print(fixed_code_1)

In [ ]:
# Example 2: API Usage Error
print("\n=== Example 2: API Usage Error ===")
traceback_2 = """Traceback (most recent call last):
  File "test.py", line 5, in <module>
    model = AutoModel.from_pretrained('invalid-model-xyz')
  File "/transformers/modeling_utils.py", line 123, in from_pretrained
    raise EnvironmentError(f"Model {model_name} not found")
EnvironmentError: Model invalid-model-xyz not found"""

code_2 = """from transformers import AutoModel

def load_model():
    # Load pretrained model
    model = AutoModel.from_pretrained('invalid-model-xyz')
    return model
"""

error_info_2 = parser.parse_traceback(traceback_2)
fixes_2 = analyzer.analyze_error(error_info_2, code_2)

print(f"Error: {error_info_2.error_type} - {error_info_2.error_message}")
print(f"\nTop Fix: {fixes_2[0].description}")
print(f"Code Change: {fixes_2[0].code_changes[0]}")

In [ ]:
# Example 3: Type Error
print("\n=== Example 3: Type Error ===")
traceback_3 = """Traceback (most recent call last):
  File "test.py", line 8, in process_image
    output = model(image_array)
TypeError: forward() expected Tensor but got numpy.ndarray"""

code_3 = """import numpy as np
import torch

def process_image(image_path):
    # Load and process image
    image_array = np.random.rand(224, 224, 3)
    model = load_model()
    output = model(image_array)
    return output
"""

error_info_3 = parser.parse_traceback(traceback_3)
fixes_3 = analyzer.analyze_error(error_info_3, code_3)

fixed_code_3, applied_3 = regenerator.regenerate_with_fixes(
    code_3, error_info_3, "Process image with model"
)

print(f"Applied fixes: {applied_3}")
print("\nModified section:")
print("output = model(torch.tensor(image_array))  # Convert to tensor")

## 7. Error Analysis Statistics and Visualization

In [ ]:
# Simulate error analysis on multiple code samples
error_samples = [
    {"type": "NameError", "category": ErrorCategory.IMPORT, "fixed": True, "iterations": 1},
    {"type": "ImportError", "category": ErrorCategory.IMPORT, "fixed": True, "iterations": 1},
    {"type": "TypeError", "category": ErrorCategory.TYPE, "fixed": True, "iterations": 2},
    {"type": "EnvironmentError", "category": ErrorCategory.API_USAGE, "fixed": True, "iterations": 2},
    {"type": "FileNotFoundError", "category": ErrorCategory.RESOURCE, "fixed": True, "iterations": 1},
    {"type": "RuntimeError", "category": ErrorCategory.RUNTIME, "fixed": False, "iterations": 3},
    {"type": "ValueError", "category": ErrorCategory.RUNTIME, "fixed": True, "iterations": 2},
    {"type": "AttributeError", "category": ErrorCategory.API_USAGE, "fixed": True, "iterations": 2},
]

# Convert to DataFrame
error_df = pd.DataFrame(error_samples)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Error categories distribution
ax1 = axes[0, 0]
category_counts = error_df['category'].apply(lambda x: x.value).value_counts()
ax1.pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%')
ax1.set_title('Distribution of Error Categories')

# 2. Fix success rate by category
ax2 = axes[0, 1]
fix_rate = error_df.groupby('category')['fixed'].mean()
ax2.bar([c.value for c in fix_rate.index], fix_rate.values)
ax2.set_xlabel('Error Category')
ax2.set_ylabel('Fix Success Rate')
ax2.set_title('Fix Success Rate by Category')
ax2.tick_params(axis='x', rotation=45)

# 3. Iterations needed for fix
ax3 = axes[1, 0]
fixed_errors = error_df[error_df['fixed']]
ax3.hist(fixed_errors['iterations'], bins=range(1, 5), edgecolor='black', alpha=0.7)
ax3.set_xlabel('Iterations Needed')
ax3.set_ylabel('Count')
ax3.set_title('Number of Iterations to Fix Errors')

# 4. Error type frequency
ax4 = axes[1, 1]
error_type_counts = error_df['type'].value_counts()
ax4.barh(error_type_counts.index, error_type_counts.values)
ax4.set_xlabel('Frequency')
ax4.set_ylabel('Error Type')
ax4.set_title('Frequency of Error Types')

plt.tight_layout()
plt.show()

## 8. Building Error Knowledge Base

In [ ]:
class ErrorKnowledgeBase:
    """Maintain knowledge base of errors and successful fixes"""
    
    def __init__(self):
        self.knowledge = defaultdict(list)
        self.success_patterns = []
    
    def add_error_resolution(self, error_info: ErrorInfo, fix: ErrorFix, success: bool):
        """Add error-fix pair to knowledge base"""
        entry = {
            "error_pattern": f"{error_info.error_type}: {error_info.error_message}",
            "category": error_info.category.value,
            "fix_description": fix.description,
            "fix_confidence": fix.confidence,
            "success": success,
            "timestamp": pd.Timestamp.now()
        }
        
        self.knowledge[error_info.category].append(entry)
        
        if success:
            self.success_patterns.append({
                "pattern": error_info.error_message,
                "fix": fix
            })
    
    def get_similar_fixes(self, error_info: ErrorInfo, threshold: float = 0.7) -> List[ErrorFix]:
        """Find similar successful fixes from knowledge base"""
        similar_fixes = []
        
        # Search in same category
        category_knowledge = self.knowledge.get(error_info.category, [])
        
        for entry in category_knowledge:
            if entry["success"] and self._similarity(error_info.error_message, 
                                                    entry["error_pattern"]) > threshold:
                similar_fixes.append({
                    "fix": entry["fix_description"],
                    "confidence": entry["fix_confidence"] * 0.9,  # Slightly lower confidence
                    "source": "knowledge_base"
                })
        
        return similar_fixes
    
    def _similarity(self, str1: str, str2: str) -> float:
        """Simple string similarity (Jaccard similarity)"""
        set1 = set(str1.lower().split())
        set2 = set(str2.lower().split())
        
        if not set1 or not set2:
            return 0.0
        
        intersection = set1.intersection(set2)
        union = set1.union(set2)
        
        return len(intersection) / len(union)
    
    def generate_report(self) -> str:
        """Generate knowledge base report"""
        report = ["Error Knowledge Base Report"]
        report.append("=" * 40)
        
        for category, entries in self.knowledge.items():
            success_count = sum(1 for e in entries if e["success"])
            total_count = len(entries)
            
            report.append(f"\n{category.value}:")
            report.append(f"  Total errors: {total_count}")
            report.append(f"  Successfully fixed: {success_count}")
            report.append(f"  Success rate: {success_count/total_count*100:.1f}%")
        
        report.append(f"\nTotal unique fix patterns: {len(self.success_patterns)}")
        
        return "\n".join(report)

# Demo knowledge base
kb = ErrorKnowledgeBase()

# Add some sample resolutions
sample_errors = [
    (error_info_1, fixes_1[0], True),
    (error_info_2, fixes_2[0], True),
    (error_info_3, fixes_3[0], True),
]

for error, fix, success in sample_errors:
    kb.add_error_resolution(error, fix, success)

print(kb.generate_report())

## Key Takeaways

1. **Error Analysis là Core Component**:
   - Không chỉ detect error mà còn understand root cause
   - Categorization giúp apply targeted fixes
   - Pattern matching với knowledge base tăng efficiency

2. **Multi-level Analysis**:
   - Syntax level: Parse traceback structure
   - Semantic level: Understand error meaning
   - Domain level: Apply AI library knowledge

3. **Iterative Improvement**:
   - Each error provides learning opportunity
   - Knowledge base grows over time
   - Fix confidence improves with experience

4. **Integration with Code Generation**:
   - Error analysis feeds back to prompt improvement
   - Specific fixes guide code regeneration
   - Reduces iterations needed for working code

5. **Practical Applications**:
   - Can be extended to any programming domain
   - Knowledge base can be shared across projects
   - Enables automated debugging assistance